In [18]:
#Using Clustering for Collaborative Filtering 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

books =pd.read_csv ('Data/BX-Books.csv', sep = ";", on_bad_lines = 'skip', encoding = 'latin-1')
users =pd.read_csv ('Data/BX-Users.csv', sep = ";", on_bad_lines = 'skip', encoding = 'latin-1')
ratings =pd.read_csv ('Data/BX-Book-Ratings.csv', sep = ";", on_bad_lines = 'skip', encoding = 'latin-1')

/var/folders/1b/_xjhfjcj79sfhhwtpgm9h2v40000gn/T/ipykernel_46132/148238669.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books =pd.read_csv ('Data/BX-Books.csv', sep = ";", on_bad_lines = 'skip', encoding = 'latin-1')


Section 1: Cleaning the Data

In [19]:
books.columns
books.rename(columns = {"Book-Title": "title", "Book-Author": "author", "Year-Of-Publication":"year", "Publisher": "publisher", "Image-URL-L": "url"}, inplace =True)
books.head()

,ISBN,title,author,year,publisher,Image-URL-S,Image-URL-M,url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [20]:
ratings.columns
ratings.rename (columns = {"User-ID":"id","isbn":"ISBN","Book-Rating":'rating'}, inplace = True)

#Displaying how many books a certain person has rated
ratings['id'].value_counts()

#Counting how many users have rated books
ratings['id'].unique().shape

#Making a variable to consider quantity of books needed to be rated to consider a reviewer credible.
bkrtd = 200

#Only conisdering users who have rated many books.
true_reviewers=ratings['id'].value_counts() > bkrtd

#Count of users that have reviewed many books
true_reviewers[true_reviewers].shape

user_index = true_reviewers[true_reviewers].index

#Filtering ratings dataframe to only contain the users who have reviewed many books.
ratings = ratings[ratings['id'].isin(user_index)]

In [21]:
ratings_with_books = ratings.merge (books, on="ISBN")
ratings_with_books.head()


,id,ISBN,rating,title,author,year,publisher,Image-URL-S,Image-URL-M,url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...


In [22]:
num_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()
num_rating.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [23]:
num_rating.rename(columns = {'rating':'num_of_rating'}, inplace=True)
num_rating.head()


,title,num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [24]:
ratings_with_books = ratings_with_books.merge(num_rating, on = "title")

ratings_with_books.head()

,id,ISBN,rating,title,author,year,publisher,Image-URL-S,Image-URL-M,url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82


In [25]:
popular = ratings_with_books[ratings_with_books['num_of_rating']>=50]
popular.head()

,id,ISBN,rating,title,author,year,publisher,Image-URL-S,Image-URL-M,url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82


In [26]:
#popular.drop(columns = ["num_of_rating_x","num_of_rating_y","Image-URL-S","Image-URL-M"], inplace = True)
popular.sample(10)


,id,ISBN,rating,title,author,year,publisher,Image-URL-S,Image-URL-M,url,num_of_rating
8249,106225,0446602620,0,Blood Work,Michael Connelly,1998,Warner Books,http://images.amazon.com/images/P/0446602620.0...,http://images.amazon.com/images/P/0446602620.0...,http://images.amazon.com/images/P/0446602620.0...,54
19227,125519,0440213525,10,The Client,John Grisham,1994,Dell Publishing Company,http://images.amazon.com/images/P/0440213525.0...,http://images.amazon.com/images/P/0440213525.0...,http://images.amazon.com/images/P/0440213525.0...,183
2024,251613,0316182540,0,White Oleander : A Novel,Janet Fitch,2001,"Little, Brown",http://images.amazon.com/images/P/0316182540.0...,http://images.amazon.com/images/P/0316182540.0...,http://images.amazon.com/images/P/0316182540.0...,116
172254,76352,0446354732,0,Nothing Lasts Forever,Sidney Sheldon,1995,Warner Books,http://images.amazon.com/images/P/0446354732.0...,http://images.amazon.com/images/P/0446354732.0...,http://images.amazon.com/images/P/0446354732.0...,51
2515,6543,0316777730,0,Naked,David Sedaris,1998,Back Bay Books,http://images.amazon.com/images/P/0316777730.0...,http://images.amazon.com/images/P/0316777730.0...,http://images.amazon.com/images/P/0316777730.0...,53
17791,127429,0679419454,0,Disclosure,Michael Crichton,1994,Alfred A. Knopf,http://images.amazon.com/images/P/0679419454.0...,http://images.amazon.com/images/P/0679419454.0...,http://images.amazon.com/images/P/0679419454.0...,122
18633,124079,0425147363,0,Tom Clancy's Op-Center (Tom Clancy's Op Center...,Tom Clancy,1995,Berkley Publishing Group,http://images.amazon.com/images/P/0425147363.0...,http://images.amazon.com/images/P/0425147363.0...,http://images.amazon.com/images/P/0425147363.0...,63
92876,110912,0553574663,0,The Grilling Season,Diane Mott Davidson,1998,Bantam Books,http://images.amazon.com/images/P/0553574663.0...,http://images.amazon.com/images/P/0553574663.0...,http://images.amazon.com/images/P/0553574663.0...,50
121505,162052,0449003787,10,"\O\"" Is for Outlaw""",SUE GRAFTON,2001,Ballantine Books,http://images.amazon.com/images/P/0449003787.0...,http://images.amazon.com/images/P/0449003787.0...,http://images.amazon.com/images/P/0449003787.0...,105
107717,87555,0375700757,0,Cold Mountain : A Novel,CHARLES FRAZIER,1998,Vintage,http://images.amazon.com/images/P/0375700757.0...,http://images.amazon.com/images/P/0375700757.0...,http://images.amazon.com/images/P/0375700757.0...,92


In [27]:
popular.drop_duplicates (['id','title'],inplace = True)
popular.shape

/var/folders/1b/_xjhfjcj79sfhhwtpgm9h2v40000gn/T/ipykernel_46132/1921673419.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popular.drop_duplicates (['id','title'],inplace = True)


(59850, 11)

Creating a Pivot Table to transition into Clustering

In [28]:
book_pivot = popular.pivot_table(columns='id',index='title',values = 'rating')
book_pivot.fillna(0,inplace=True)
print(book_pivot)

id                                                  254     2276    2766    \
title                                                                        
1984                                                   9.0     0.0     0.0   
1st to Die: A Novel                                    0.0     0.0     0.0   
2nd Chance                                             0.0    10.0     0.0   
4 Blondes                                              0.0     0.0     0.0   
84 Charing Cross Road                                  0.0     0.0     0.0   
...                                                    ...     ...     ...   
Year of Wonders                                        0.0     0.0     0.0   
You Belong To Me                                       0.0     0.0     0.0   
Zen and the Art of Motorcycle Maintenance: An I...     0.0     0.0     0.0   
Zoya                                                   0.0     0.0     0.0   
\O\" Is for Outlaw"                                    0.0     0

Start of Model Training

In [29]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

book_sparse = csr_matrix(book_pivot)
model = NearestNeighbors(algorithm='brute')

model.fit(book_sparse)

distance, suggestion = model.kneighbors (book_pivot.iloc [237,:].values.reshape(1,-1),n_neighbors = 6)

book_title = book_pivot.index



for i in range(len(suggestion)):
    print(book_title[suggestion[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [30]:
popular.head()

,id,ISBN,rating,title,author,year,publisher,Image-URL-S,Image-URL-M,url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82


Creating Artifacts for a potential Web App

In [31]:
import pickle

pickle.dump(model,open('artifacts/model.pkl','wb'))
pickle.dump(book_title,open('artifacts/book_title.pkl','wb'))
pickle.dump(popular,open('artifacts/popular_books.pkl','wb'))
pickle.dump(book_pivot,open('artifacts/book_pivot.pkl','wb'))

Creating A Method for the Model

In [32]:
def recommend_book(book_title):
    book_id = np.where(book_pivot.index == book_title)[0][0] #Determining the index of the input book title
    distance, suggestion = model.kneighbors (book_pivot.iloc [book_id,:].values.reshape(1,-1),n_neighbors = 5) #Finding Similar books and recommending them trhough clustering
    for i in range (len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        for j in books:
            if j != book_title:
                print (j)


book_pivot.sample(100)
print(book_pivot.query('title.str.contains ("Diary")',engine='python'))           

id                                                254     2276    2766    \
title                                                                      
Anne Frank: The Diary of a Young Girl                0.0     0.0     0.0   
Blue Diary                                           0.0     0.0     0.0   
Bridget Jones's Diary                                0.0     0.0     0.0   
Suzanne's Diary for Nicholas                         0.0     0.0     0.0   
The Diary of Ellen Rimbauer: My Life at Rose Red     0.0     0.0     0.0   

id                                                2977    3363    3757    \
title                                                                      
Anne Frank: The Diary of a Young Girl                0.0     0.0     0.0   
Blue Diary                                           0.0     0.0     0.0   
Bridget Jones's Diary                                0.0     0.0     0.0   
Suzanne's Diary for Nicholas                         0.0     0.0     0.0   
The Diary o

### Testing Method

In [33]:
book_name = 'The Two Towers (The Lord of the Rings, Part 2)'
recommend_book (book_name)

The Return of the King (The Lord of the Rings, Part 3)
The Bourne Supremacy
Pleading Guilty
Alaska


### Making Recommendations using SVD Matrix Factorization

In [34]:
print(book_pivot)

id                                                  254     2276    2766    \
title                                                                        
1984                                                   9.0     0.0     0.0   
1st to Die: A Novel                                    0.0     0.0     0.0   
2nd Chance                                             0.0    10.0     0.0   
4 Blondes                                              0.0     0.0     0.0   
84 Charing Cross Road                                  0.0     0.0     0.0   
...                                                    ...     ...     ...   
Year of Wonders                                        0.0     0.0     0.0   
You Belong To Me                                       0.0     0.0     0.0   
Zen and the Art of Motorcycle Maintenance: An I...     0.0     0.0     0.0   
Zoya                                                   0.0     0.0     0.0   
\O\" Is for Outlaw"                                    0.0     0

In [60]:
from sklearn.decomposition import TruncatedSVD    
SVD = TruncatedSVD(n_components=12, random_state=17)
matrix = SVD.fit_transform(book_pivot)
pickle.dump(matrix,open('artifacts/SVD_matrix.pkl','wb'))

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [52]:
corr = np.corrcoef(matrix)
books = book_pivot.T
books_cols = book_pivot.T.columns
books_list=list(books_cols)
Zoya = books_list.index('Zoya')


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [53]:
corr_Zoya = corr[Zoya]
similar_books = list(books_cols[(corr_Zoya<1.0)&(corr_Zoya>0.75)])

print(similar_books)

['1st to Die: A Novel', 'A Child Called \\It\\": One Child\'s Courage to Survive"', 'Accident', 'All Around the Town', 'Along Came a Spider (Alex Cross Novels)', 'Before I Say Good-Bye', 'Bridget Jones: The Edge of Reason', 'Cat &amp; Mouse (Alex Cross Novels)', 'Cruel &amp; Unusual (Kay Scarpetta Mysteries (Paperback))', 'Daddy', "Daddy's Little Girl", 'Dawn (Cutler)', 'Exclusive', 'Eyes of a Child', 'Family Album', 'Fine Things', 'Five Days in Paris', 'GARDEN OF SHADOWS (Dollanger Saga (Paperback))', 'Granny Dan', 'Heartbeat', 'Hide &amp; Seek', 'Homecoming', 'I Know Why the Caged Bird Sings', "I'll Be Seeing You", 'If Only It Were True', 'Jack &amp; Jill (Alex Cross Novels)', 'Kaleidoscope', 'Let Me Call You Sweetheart', 'Lightning', 'Loves Music, Loves to Dance', 'Message from Nam', 'Mirror Image', 'Moonlight Becomes You', 'My Gal Sunday', 'Night', 'No Greater Love', 'On the Street Where You Live', "Pretend You Don't See Her", 'Remember Me', 'Secrets', 'Skipping Christmas', 'Stillw

In [62]:
def sim_books_scores(book_title):
    book = books_list.index(book_title)
    corr_book = corr[book] 
    # Create a DataFrame with book titles and scores
    df = pd.DataFrame({'title': books_cols, 'score': corr_book})
    # Filter for scores between 0.75 and 1
    similar = df[(df['score'] < 1.0) & (df['score'] > 0.75)]
    similar = similar.sort_values(by='score', ascending=False)
    return similar

In [73]:
results = sim_books_scores('The Hobbit : The Enchanting Prelude to The Lord of the Rings')
list(results['title'])

['The Fellowship of the Ring (The Lord of the Rings, Part 1)',
 "The Hitchhiker's Guide to the Galaxy",
 'A Year in Provence',
 'The Return of the King (The Lord of the Rings, Part 3)',
 '1984',
 'Matilda',
 'The Hundred Secret Senses',
 'Fahrenheit 451',
 'The Joy Luck Club',
 'Where the Red Fern Grows',
 'Naked',
 'The Two Towers (The Lord of the Rings, Part 2)',
 'Artemis Fowl (Artemis Fowl, Book 1)',
 'A Wrinkle in Time']

In [38]:
# #Miscelaneous Hugging Face Testing
# from transformers import pipeline

# def get_chat (model_name,input_text):
#   #Load the specified model
#   model = pipeline ("text-generation", model=model_name)

#   #Get the response from the model
#   response = model(input_text, max_length = 50)[0] ['generated_text']

#   return response

# def prompt_user():
#   model_name = input("Enter the Hugging Face Transformer model name: ")
#   input_text = input ("Enter the text to be sent to the model: ")
#   response = get_chat (model_name, input_text)
#   print("Response:", response)

# prompt_user()

ModuleNotFoundError: No module named 'transformers'

: 